In [1]:
import numpy as np

from nltk import word_tokenize
import nltk
from collections import Counter
stop_words = nltk.corpus.stopwords.words('english')

from urllib import request

In [2]:
tromso_description = """Tromsø is a beautiful city between FJORDS, ISLANDS AND MOUNTAINS, with a visible past, a fascinating history, a lively, colourful city centre, an inclusive nightlife and numerous attractions. Use the city as a base to foray into Arctic wilderness chasing Midnight Sun and Northern Lights. 01. 02."""
oslo_description = """Oslo is considered as a global city and is the major Norwegian hub for trading, shipping and banking. Location of Oslo: OSLO IS POSITIONED AT THE NORTHERNMOST END OF THE OSLOFJORD and occupies around 40 big and small islands within its limits. The climate of the region is temperate, humid."""
mining_course_description = """The aim of the course is to introduce the students to the concepts and techniques of natural languages processing and analysis, unstructured information analysis and management for better decision- making by deriving valuable insights from enterprise content regardless of source or format. The course provides deep and rich knowledge of text analysis techniques and applications including sentiment analysis and opinion mining, information access and text mining, document classification, topic extraction and other techniques and applications using real-world data and cases."""

In [3]:
def prepText(text, tokenName):
    tokenName = word_tokenize(text)
    tokenName = [token.lower() for token in tokenName]
    tokenName = [token.strip() for token in tokenName]
    tokenName = [word for word in tokenName if word not in stop_words]
    tokenName = [word for word in tokenName if word.isalnum()]
    return tokenName

In [4]:
token_tromso = prepText(tromso_description, 'token_tromso')
token_oslo = prepText(oslo_description, 'token_oslo')
token_nlp =  prepText(mining_course_description, 'token_nlp')

print(token_tromso[:10])
print(token_oslo[:10])
print(token_nlp[:10])

['tromsø', 'beautiful', 'city', 'fjords', 'islands', 'mountains', 'visible', 'past', 'fascinating', 'history']
['oslo', 'considered', 'global', 'city', 'major', 'norwegian', 'hub', 'trading', 'shipping', 'banking']
['aim', 'course', 'introduce', 'students', 'concepts', 'techniques', 'natural', 'languages', 'processing', 'analysis']


In [5]:
url = "https://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')
print(raw[:200])

token_CP = prepText(raw, 'token_CP')
print(token_CP[:10])

The Project Gutenberg eBook of Crime and Punishment, by Fyodor Dostoevsky

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with a
['project', 'gutenberg', 'ebook', 'crime', 'punishment', 'fyodor', 'dostoevsky', 'ebook', 'use', 'anyone']
